In [1]:
import sys
sys.path.insert(0, "../") 

import algorithms
import time
import numpy as np
import torch
from data_generators import *
import pandas as pd

import matplotlib.pyplot as plt 

from experiment_helper import plot_wtp_distribution, save_results, plot_loss, save_matrix
import sys
from metrics_helper import MetricsHelper
from IPython.display import display, HTML

# Setting: Small vs. Medium vs. Big

In [2]:
#Good M-sized config
nb_session = 3
nb_items_session = 10
nb_users = 1000
nb_prods = 100
dimension = 10

In [3]:
#Hard L-sized config
#nb_session = 15
#nb_items_session = 2
#nb_users = 1000
#nb_prods = 100
#dimension = 10

## Data Generation

In [4]:
gap = 0

count = 0 

#We make sure that we generate a dataset in which showing the most popular products is not the best thing to do
while gap < 0.12:

    data_generator = DataGenerator(nb_prods, nb_users, nb_items_session, variety = 3, nb_session=nb_session, dimension=dimension)
    (nb_sess, nb_sales, nb_buyers) = data_generator.get_stats()
    mh = MetricsHelper(data_generator)

    print("Nb_sessions: " + str(nb_sess))
    print("Nb_events: " + str(nb_sess * nb_items_session))
    print("Nb_sales: " + str(nb_sales))
    print("Nb_buyers: " + str(nb_buyers) + "\n")

    metrics_helper = MetricsHelper(data_generator)
    train_surplus = metrics_helper.get_sessions_surplus("training")[0] / nb_users
    print("Avg. Training Payoff: " + str(train_surplus))

    def find_best_item(matrix):
        max_val = np.amax(matrix,1)
        arg_max = np.argmax(matrix, axis=1)
        max_val = np.where(max_val < 0, 0, max_val)
        return (max_val, arg_max)

    true_payoff = data_generator.payoff_matrix
    true_wtp = data_generator.wtp_matrix
    (true_best_vals,true_best_items) = find_best_item(true_payoff)
    best_surplus = np.sum(true_best_vals)/nb_users

    algo0 = algorithms.BestOf_model(data_generator, comment="test tensorboard")
    ranking = np.tile(algo0.topk_choice, (nb_users,1))
    bof_surplus, bof_welfare, bof_sales, bof_precision = mh.BannerPerformance(ranking,10)

    print("Avg. Best Payoff: " + str(best_surplus))
    print("Avg. BestOf10 Payoff: " + str(bof_surplus))
    
    gap = (best_surplus - bof_surplus)/best_surplus
    
    count += 1
    print("Attempt #" + str(count) + " Gap: " + str(gap) + "\n\n")


exp_dict = {}
exp_dict["run_id"] = int(time.time())

exp_dict["env"]={}
exp_dict["env"]["nb_sessions"]=nb_session
exp_dict["env"]["nb_items_session"]=nb_items_session
exp_dict["env"]["nb_users"]=nb_users
exp_dict["env"]["nb_prods"]=nb_prods
exp_dict["env"]["dimension"]=dimension

exp_dict["algos"] = {}
exp_dict["algos"]["skyline"]={}
exp_dict["algos"]["skyline"]["payoff"] = best_surplus
exp_dict["algos"]["skyline"]["precision"] = 1

exp_dict["algos"]["baseline"]={}
exp_dict["algos"]["baseline"]["payoff"] = train_surplus
exp_dict["algos"]["baseline"]["precision"] = "NA"

Nb_sessions: 3000
Nb_events: 30000
Nb_sales: 2118
Nb_buyers: 925

Avg. Training Payoff: 9.313093870416939
Avg. Best Payoff: 14.299696209107147
Avg. BestOf10 Payoff: 12.427211645031187
Attempt #1 Gap: 0.13094575833599997




## Define helper functions

In [5]:
def get_matrix(users, items):
    users = users.detach().numpy()
    items = items.detach().numpy()
    matrix = np.array([[np.dot(users[i_user], items[i_item])
                        for i_user in range(len(users))] for i_item in range(len(items))])
    return matrix
    
def heatmap2d(arr: np.ndarray):
    f = plt.figure(figsize=(10,3))
    plt.imshow(arr, cmap='viridis')
    plt.colorbar()
    plt.show()
    
def find_topk_items(matrix, k):
    top_k = np.argsort(-matrix,axis=1)
    top_val = np.take_along_axis(matrix, top_k, axis=1)[:,:k]
    top_k = top_k[:,:k]
    return (top_val, top_k)

def std_matrix(A):
    A = (A - np.mean(A, axis=0)) / np.std(A, axis=0)
    return A

def generateRankings(users, items, elasticities, prices, weight):
    predicted_wtp = np.dot(users,np.transpose(items))
    price_impact = np.outer(elasticities, prices)
    predicted_payoff = predicted_wtp - price_impact
    pSale = 1 / (1 + np.exp(-predicted_payoff))
    wpSale = np.multiply(pSale,weight)
    rankedPreferences = np.argsort(-wpSale,axis=1)
    return rankedPreferences

def perfect_ranking(true_payoff, prices):
    true_welfare = true_payoff + prices
    payoff_ranking = np.argsort(-true_payoff,axis=1)
    welfare_ranking = np.argsort(-true_welfare,axis=1)
    return payoff_ranking, welfare_ranking


def cap_elasticities(elasticities, nb_prods):
    nb_users = elasticities.shape[0]
    min_elasticity = np.ones(nb_users)*0.1
    elasticities2 = np.where(elasticities < 0.1, min_elasticity, elasticities)
    #print(elasticities2.shape)
    elasticities3 = np.tile(elasticities2, (nb_prods,1)).transpose()
    #print(elasticities3.shape)
    return elasticities2, elasticities3


def computeWeightMatrix(users, items, elasticities, prices, use_price, matrix_type):
    nb_users = users.shape[0]
    nb_prods = items.shape[0]

    elasticities_cap, elasticitiesMat = cap_elasticities(elasticities, nb_prods)
    predicted_wtp = np.dot(users,np.transpose(items))/elasticitiesMat
    price_impact = np.outer(elasticities,prices)
    predicted_payoff = np.dot(users,np.transpose(items)) - price_impact
    
    if matrix_type == "sales":    
        return np.ones((nb_users, nb_prods))
        
    if matrix_type == "welfare":
        if use_price == 0:
            return predicted_wtp + price_impact
        return predicted_wtp
    
    if matrix_type == "payoff":
        if use_price == 0:
            return predicted_wtp
        return predicted_payoff
    
    if matrix_type == "margin":
        pricesMat = np.tile(prices, (nb_users, 1))
        return pricesMat
    
    
def computeTrueWeightMatrix(true_payoff, prices, matrix_type):
    nb_users = true_payoff.shape[0]
    nb_prods = true_payoff.shape[1]
    
    if matrix_type == "sales":    
        return np.ones((nb_users, nb_prods))
        
    if matrix_type == "welfare":
        return true_payoff + prices
    
    if matrix_type == "payoff":
        return true_payoff
    
    if matrix_type == "margin":
        pricesMat = np.tile(prices, (nb_users, 1))
        return pricesMat

    
def ranking_analysis(rankings, algo_name, objective, topk, exp_list):
    
    surplus, welfare, sales, precision  = mh.BannerPerformance(rankings,topk)

    curr_exp_list = []
    curr_exp_list.append([algo_name, "NA", "NA", objective, topk, surplus, welfare, welfare-surplus, sales, precision, "NA"])

    exp_list.append(curr_exp_list)
    df = pd.DataFrame(curr_exp_list, columns=['Algo', 'Crit', 'UsePrice', 'Objective','k','Utility@k','Welfare@k','Revenue@k','Sales@k','Precision@k','L2'])
    display(df)
    
    return exp_list


def algo_list_analysis(algo_list, algo_stats, algo_names, criterion, topk, use_bootstrap):
    exp_list = []
    csv_list = []
    
    for index in range(len(algo_list)):
        algo = algo_list[index]
        algo_stat = algo_stats[index]
        algo_name = algo_names[index]
        df, df2, exp_list, csv_list = algo_analysis(algo, algo_stat, algo_name, criterion, topk, use_bootstrap, exp_list, csv_list)
    
    flat_list = [item for sublist in exp_list for item in sublist]
    df = pd.DataFrame(flat_list, columns=['Algo', 'Crit', 'UsePrice', 'Objective','k','Utility@k','Welfare@k','Revenue@k','Sales@k','Precision@k','L2'])
    display(df)
    
    flat_list = [item for sublist in csv_list for item in sublist]
    df2 = pd.DataFrame(flat_list, columns=['Algo','Objective','Utility-M','Utility-V','Welfare-M','Welfare-V','Sales-M','Sales-V','Precision-M','Precision-V','Revenue-M','Revenue-V'])
    display(df2)
    return df, df2
     
    
def algo_analysis(algo, algo_stats, algo_name, criterion, topk, use_bootstraps, exp_list, csv_list):

    #######################
    ###Plot training curves
    #######################
    c = algo_stats
    f = plt.figure(figsize=(10,3))
    ax = f.add_subplot(141)
    ax2 = f.add_subplot(142)
    ax3 = f.add_subplot(143)
    ax4 = f.add_subplot(144)

    ax.plot(c.llh_train,color="red",label="train")
    ax.plot(c.llh_validation,color="blue",label="validation")
    ax2.plot(c.L2,color="grey",label="L2")
    ax3.plot(c.SurplusAtK,color="green",label="surplus")
    ax4.plot(c.PrecisionAtK,color="orange",label="precision")

    ###################################
    ###Choose the best model checkpoint 
    ###################################
    if criterion == "last":
        argmin_valid = len(c.llh_train)-1
    if criterion == "train":
        argmin_valid = np.argmin(c.llh_train)
    if criterion == "validation":
        argmin_valid = np.argmin(c.llh_validation)
    
    users = c.user_mu[argmin_valid] 
    items = c.item_latent[argmin_valid][:nb_prods,:]
    elasticities = c.elasticity[argmin_valid]
    prices = algo._prices.detach().numpy()
    
    ##########################################
    ###Compute the predicted weighting factors
    ##########################################
    use_price = algo._use_price
    salesMat = computeWeightMatrix(users, items, elasticities, prices, use_price, "sales")
    welfareMat = computeWeightMatrix(users, items, elasticities, prices, use_price, "welfare")
    payoffMat = computeWeightMatrix(users, items, elasticities, prices, use_price, "payoff")
    marginMat = computeWeightMatrix(users, items, elasticities, prices, use_price, "margin")
    
    ######################################################
    ###Compute the ranking using various weighting schemes
    ######################################################    
    banners1 = generateRankings(users, items, elasticities, prices, salesMat)
    banners2 = generateRankings(users, items, elasticities, prices, welfareMat)
    banners3 = generateRankings(users, items, elasticities, prices, payoffMat)
    banners4 = generateRankings(users, items, elasticities, prices, marginMat)

    curr_exp_list = []
    csv_exp_list = []
    
    l2 = c.L2[argmin_valid]
        
    if use_bootstraps == 0:
        surplus1, welfare1, sales1, precision1  = mh.BannerPerformance(banners1,topk)
        surplus2, welfare2, sales2, precision2  = mh.BannerPerformance(banners2,topk)
        surplus3, welfare3, sales3, precision3  = mh.BannerPerformance(banners3,topk)
        surplus4, welfare4, sales4, precision4  = mh.BannerPerformance(banners4,topk)
        
        margin1 = welfare1- surplus1
        margin2 = welfare2- surplus2
        margin3 = welfare3- surplus3
        margin4 = welfare4- surplus4
    
    else:
        
        nb_bootstraps = 100

        metrics_mean_vct1, metrics_std_vct1  = mh.BannerPerformanceWithBootstraps(banners1,nb_bootstraps,topk)
        metrics_mean_vct2, metrics_std_vct2  = mh.BannerPerformanceWithBootstraps(banners2,nb_bootstraps,topk)
        metrics_mean_vct3, metrics_std_vct3  = mh.BannerPerformanceWithBootstraps(banners3,nb_bootstraps,topk)
        metrics_mean_vct4, metrics_std_vct4  = mh.BannerPerformanceWithBootstraps(banners4,nb_bootstraps,topk)
        
        surplus1 = str(metrics_mean_vct1[0]) + "+/-" + str(metrics_std_vct1[0])
        welfare1 = str(metrics_mean_vct1[1]) + "+/-" + str(metrics_std_vct1[1])
        sales1 = str(metrics_mean_vct1[2]) + "+/-" + str(metrics_std_vct1[2])
        precision1 = str(metrics_mean_vct1[3]) + "+/-" + str(metrics_std_vct1[3])
        margin1 = str(metrics_mean_vct1[4]) + "+/-" + str(metrics_std_vct1[4])

        surplus2 = str(metrics_mean_vct2[0]) + "+/-" + str(metrics_std_vct2[0])
        welfare2 = str(metrics_mean_vct2[1]) + "+/-" + str(metrics_std_vct2[1])
        sales2 = str(metrics_mean_vct2[2]) + "+/-" + str(metrics_std_vct2[2])
        precision2 = str(metrics_mean_vct2[3]) + "+/-" + str(metrics_std_vct2[3])
        margin2 = str(metrics_mean_vct2[4]) + "+/-" + str(metrics_std_vct2[4])

        surplus3 = str(metrics_mean_vct3[0]) + "+/-" + str(metrics_std_vct3[0])
        welfare3 = str(metrics_mean_vct3[1]) + "+/-" + str(metrics_std_vct3[1])
        sales3 = str(metrics_mean_vct3[2]) + "+/-" + str(metrics_std_vct3[2])
        precision3 = str(metrics_mean_vct3[3]) + "+/-" + str(metrics_std_vct3[3])
        margin3 = str(metrics_mean_vct3[4]) + "+/-" + str(metrics_std_vct3[4])

        surplus4 = str(metrics_mean_vct4[0]) + "+/-" + str(metrics_std_vct4[0])
        welfare4 = str(metrics_mean_vct4[1]) + "+/-" + str(metrics_std_vct4[1])
        sales4 = str(metrics_mean_vct4[2]) + "+/-" + str(metrics_std_vct4[2])
        precision4 = str(metrics_mean_vct4[3]) + "+/-" + str(metrics_std_vct4[3])
        margin4 = str(metrics_mean_vct4[4]) + "+/-" + str(metrics_std_vct4[4])
        
        m1 = metrics_mean_vct1
        v1 = metrics_std_vct1
        csv_exp_list.append([algo_name,"sales", m1[0],v1[0], m1[1],v1[1], m1[2],v1[2], m1[3],v1[3], m1[4],v1[4]])
                          
        m2 = metrics_mean_vct2
        v2 = metrics_std_vct2 
        csv_exp_list.append([algo_name,"welfare", m2[0],v2[0], m2[1],v2[1], m2[2],v2[2], m2[3],v2[3], m2[4],v2[4]])
                         
        m3 = metrics_mean_vct3
        v3 = metrics_std_vct3                      
        csv_exp_list.append([algo_name,"utility", m3[0],v3[0], m3[1],v3[1], m3[2],v3[2], m3[3],v3[3], m3[4],v3[4]])
                         
        m4 = metrics_mean_vct4
        v4 = metrics_std_vct4                      
        csv_exp_list.append([algo_name,"revenue", m4[0],v4[0], m4[1],v4[1], m4[2],v4[2], m4[3],v4[3], m4[4],v4[4]])
 
    
    curr_exp_list.append([algo_name, criterion, use_price, "sales", topk, surplus1, welfare1, margin1, sales1, precision1, l2])
    curr_exp_list.append([algo_name, criterion, use_price, "welfare", topk, surplus2, welfare2, margin2, sales2, precision2, l2])
    curr_exp_list.append([algo_name, criterion, use_price, "utility", topk, surplus3, welfare3, margin3, sales3, precision3, l2])
    curr_exp_list.append([algo_name, criterion, use_price, "revenue", topk, surplus4, welfare4, margin4, sales4, precision4, l2])

    exp_list.append(curr_exp_list)
    df = pd.DataFrame(curr_exp_list, columns=['Algo', 'Crit', 'UsePrice', 'Objective','k','Payoff@k','Welfare@k','Margin@k','Sales@k','Precision@k','L2'])
    #display(df)
      
    csv_list.append(csv_exp_list)
    df2 = pd.DataFrame(csv_exp_list, columns=['Algo','Objective','Payoff-M','Payoff-V','Welfare-M','Welfare-V','Sales-M','Sales-V','Precision-M','Precision-V','Margin-M','Margin-V'])
    #display(df2)    
       
    return (df, df2, exp_list, csv_list)




## Optimization config

In [6]:
n_epochs = 251
batch_size = 100
learning_rate = 1e-1
weight_decay = 1e-4
l2_lambda = 6 #1e-1
topk =10

exp_dict["config"]={}
exp_dict["config"]["nb_epochs"]=n_epochs
exp_dict["config"]["batch_size"]=batch_size
exp_dict["config"]["learning_rate"]=learning_rate

exp_list = []
csv_list = []

## BestOf & Skyline

In [7]:
payoff_ranking, welfare_ranking = perfect_ranking(true_payoff, data_generator.prices)

algo0_name = "bestof"

algo0 = algorithms.BestOf_model(data_generator, comment="test tensorboard")
ranking = np.tile(algo0.topk_choice, (nb_users,1))

exp_list = ranking_analysis(payoff_ranking, "oracle", "payoff", 10, exp_list)
exp_list = ranking_analysis(welfare_ranking, "oracle", "welfare", 10, exp_list)
exp_list = ranking_analysis(ranking, algo0_name, "sales", 10, exp_list)

,Algo,Crit,UsePrice,Objective,k,Utility@k,Welfare@k,Revenue@k,Sales@k,Precision@k,L2
0,oracle,NA,NA,payoff,10,14.299696,19.89483,5.595134,0.997,1.0,NA


,Algo,Crit,UsePrice,Objective,k,Utility@k,Welfare@k,Revenue@k,Sales@k,Precision@k,L2
0,oracle,NA,NA,welfare,10,14.299615,19.897502,5.597887,0.997,0.999,NA


,Algo,Crit,UsePrice,Objective,k,Utility@k,Welfare@k,Revenue@k,Sales@k,Precision@k,L2
0,bestof,NA,NA,sales,10,12.427212,17.309136,4.881925,0.981,0.59,NA


## MF-pclick

In [ ]:
algo1_name = "mf_pclick"

l2_lambda_pclick = 6 
start = time.time()
algo1 = algorithms.MF_pclick_model(data_generator,  data_generator.dimension, topk, comment="test tensorboard")
algo1_stats = algo1.train(n_epochs=n_epochs,lr=learning_rate,batch_size=batch_size,weight_decay=weight_decay,l2_lambda=l2_lambda_pclick)
end = time.time()

print("\nMF training time:" + str(end - start))

Step #0 => loss: tensor(7627.4419)
Step #25 => loss: tensor(1772.5413)
Step #50 => loss: tensor(1087.5736)
Step #75 => loss: tensor(1045.1206)
Step #100 => loss: tensor(1061.5471)
Step #125 => loss: tensor(1050.1012)
Step #150 => loss: tensor(1049.9692)
Step #175 => loss: tensor(1028.2302)
Step #200 => loss: tensor(1032.4691)
Step #225 => loss: tensor(1064.9948)


In [ ]:
(df, df2, exp_list, csv_list) = algo_analysis(algo1, algo1_stats, algo1_name, "last", 10, 1, exp_list, csv_list)

## MF-softmax

In [ ]:
algo2_name = "mf_sm"
sm_l2_lambda = 5 #1e-1

start = time.time()
algo2 = algorithms.MF_model(data_generator,  data_generator.dimension, topk, use_price=0, comment="test tensorboard")
algo2_stats = algo2.train(n_epochs=n_epochs,lr=learning_rate,batch_size=batch_size,weight_decay=weight_decay,l2_lambda=sm_l2_lambda)
end = time.time()
print("\nMF training time:" + str(end - start))

In [ ]:
(df, df2, exp_list, csv_list) = algo_analysis(algo2, algo2_stats, algo2_name, "last", 10, 1, exp_list, csv_list)

## RUM-MF

In [ ]:
algo3_name = "mf_rum"

start = time.time()
algo3 = algorithms.MF_rum_model(data_generator,  data_generator.dimension, topk, comment="test tensorboard")
algo3_stats = algo3.train(n_epochs=n_epochs,lr=learning_rate,batch_size=batch_size,weight_decay=weight_decay,l2_lambda=l2_lambda)
end = time.time()

print("\nMF training time:" + str(end - start))

In [ ]:
(df, df2, exp_list, csv_list) = algo_analysis(algo3, algo3_stats, algo3_name, "last", 10, 1, exp_list, csv_list)

## Interactive comparison

In [ ]:
payoff_ranking, welfare_ranking = perfect_ranking(true_payoff, data_generator.prices)

algo0_name = "bestof"

algo0 = algorithms.BestOf_model(data_generator, comment="test tensorboard")
ranking = np.tile(algo0.topk_choice, (nb_users,1))

exp_list = []
exp_list = ranking_analysis(payoff_ranking, "oracle", "payoff", 1, exp_list)
exp_list = ranking_analysis(welfare_ranking, "oracle", "welfare",1, exp_list)
exp_list = ranking_analysis(ranking, algo0_name, "sales", 1, exp_list)
flat_list = [item for sublist in exp_list for item in sublist]
df = pd.DataFrame(flat_list, columns=['Algo', 'Crit', 'UsePrice', 'Objective','k','Payoff@k','Welfare@k','Margin@k','Sales@k','Precision@k','L2'])
display(df)

df2 = df.drop(columns=['Crit', 'UsePrice','k','L2'])
display(df2)
#print(df2.round(2).to_latex(index=False)) 

In [ ]:
use_bootstraps = 0
topk = 1
(df, df2) = algo_list_analysis([algo3,algo2,algo1], [algo3_stats,algo3_stats,algo1_stats], ["mf-rum","mf-sm","mf-pclick"], "last", topk, use_bootstraps)

In [ ]:
print(exp_dict)